In [87]:
import requests
import re
import pandas as pd

In [64]:
def get_title_start_and_end_index(html_text):
  start = html_text.find("<li class=\"lrv-u-width-100p")
  end = html_text[start:].find('</li>')
  return (start, end)

In [70]:
def get_stat_start_and_end_index(html_text):
  start = html_text.find("<ul class=\"lrv-a-unstyle-list")
  end = html_text[start:].find('</ul>')
  return (start, end)

In [45]:
def sanitize_html(html_text):
  text = re.sub('<[^<]+?>', '', html_text)
  return text

In [54]:
def remove_tabs(text):
  text = text.replace("\t", "")
  return text

In [103]:
def process_text(text):
  text = text.replace("&#039;", "'")
  text = text.replace("&amp;", "&")
  return text

In [56]:
def extract_title_and_artists(text):
  text_list = text.splitlines()
  res = []
  for t in text_list:
    if len(t) > 0:
      res.append(t)
  return res

In [83]:
def extract_stats(text):
  text_list = text.splitlines()
  res = []
  for t in text_list:
    if len(t) > 0 and len(t) < 4:
      res.append(t)
    if len(res) == 3:
      return res
  return res

In [115]:
def get_raw(text):
  return re.sub(r'[^a-zA-Z]+', '', text).lower()

In [100]:
response = requests.get("https://www.billboard.com/charts/hot-100/2000-10-06")

In [116]:
chart_dict = {"title": [],
              "raw_title": [],
              "artists": [],
              "current_pos": [],
              "peak_pos": [],
              "wks_on_chart": []}
html_text = response.text
current_pos = 1
for i in range(200):
  start, end = get_title_start_and_end_index(html_text)
  raw_text= html_text[start:start+end]
  text = sanitize_html(raw_text)
  text = remove_tabs(text)
  text = process_text(text)

  stat_start, stat_end = get_stat_start_and_end_index(html_text)
  raw_stats = html_text[stat_start:stat_start+stat_end]
  stats = sanitize_html(raw_stats)
  stats = remove_tabs(stats)

  if len(text.strip("\n")) != 0:
    title, artists = extract_title_and_artists(text)
    _, peak_pos, wks_on_chart = extract_stats(stats)
    chart_dict["title"].append(title)
    chart_dict["raw_title"].append(get_raw(title))
    chart_dict["artists"].append(artists)
    chart_dict["current_pos"].append(current_pos)
    chart_dict["peak_pos"].append(peak_pos)
    chart_dict["wks_on_chart"].append(wks_on_chart)
    current_pos += 1

  html_text = html_text[start+end:]

In [118]:
chart = pd.DataFrame(chart_dict)

In [119]:
chart.head(5)

,title,raw_title,artists,current_pos,peak_pos,wks_on_chart
0,Music,music,Madonna,1,1,9
1,Give Me Just One Night (Una Noche),givemejustonenightunanoche,98 Degrees,2,2,8
2,Doesn't Really Matter,doesntreallymatter,Janet,3,1,17
3,"Jumpin', Jumpin'",jumpinjumpin,Destiny's Child,4,3,22
4,Kryptonite,kryptonite,3 Doors Down,5,5,27
